In [1]:
! pip install quandl

In [5]:
import quandl
import pandas as pd
import numpy as np
#from google.colab import drive
import os
import time
import urllib
import json
import csv
import requests
import io
from pathlib import Path
import random

#drive.mount('/content/gdrive')


quandl.ApiConfig.api_key = 'jsb-w4XT_q1wk2Ey_PT-'
#quandl.ApiConfig.api_version = '2015-04-09'

In [6]:
# googlepath exists for googledrive finding the files. 
# string if running on computer. Have companylist.csv in same dir. Will create
# a stock_data folder and download stocks data into that. 
googlepath = "/home/ubuntu/cs230/" 


In [7]:
#mydata = quandl.get("WIKI/VIX", rows=5)
def save(googlepath, stock_csv, output_dir, filename):
    try:
        #the output dir may not exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
    except Exception as ex:
        print('Could not create output dir')
        print(ex)
        return
    filepath = os.path.join(googlepath, output_dir, filename)
    try:
#         print(stock_csv)
        df = stock_csv
        #df = df.sort_values(by='timestamp')  
#         print(df)
        df.to_csv(filepath, index=False)
        time.sleep(10)          
    except Exception as ex:
        print('Could not open file {} to write data'.format(filepath))
        print(ex)

def try_download(symbol):
    try:
        # Keep call frequency below threshold 
        #time.sleep(10)          

        # Quandl Download
        data_EQ= quandl.get_table('SHARADAR/SEP', ticker=symbol) # Daily Equity Price
        df_EQ = pd.DataFrame(data=data_EQ)
        data_DM= quandl.get_table('SHARADAR/DAILY', ticker=symbol) # Daily Metrics
        df_DM = pd.DataFrame(data=data_DM)
        data_CO= quandl.get_table('SHARADAR/ACTIONS') # Corporate Actions
        df_CO = pd.DataFrame(data=data_CO)
        data_SF1= quandl.get_table('SHARADAR/SF1', ticker=symbol) # Core US Fundamentals
        df_SF1 = pd.DataFrame(data=data_SF1)
        
        
        # Print the Head of df_EQ y and shows data
        #df_EQ

        # Merge to build 'symbol_Basic' a dataset that merged DF_EQ and DF_DM
        symbol_Basic=pd.merge(df_EQ,df_DM,on='date',sort=True) 
        #symbol_Basic.columns

        symbol_Basic=symbol_Basic.drop(['closeunadj','lastupdated_x', 'ticker_y', 'lastupdated_y'], axis=1)
        #symbol_Basic

        # Generate a Stock Data File, Small Set of Features
        #time.sleep(10)
        #symbol_Basic.to_csv (r'gdrive/My Drive/Colab Notebooks/CS230_PROJECT/stock_data/ETSY_symbol_Basic.csv', index = True, header=True )

        # In the SF1 dataset just get the ART: trailing-twelve-months ( in 'dimension' column)
        df_SF1_ART=df_SF1.loc[df_SF1.dimension=="ART"]
        df_SF1_ART=df_SF1_ART.drop(['ticker','dimension', 'calendardate','reportperiod', 'lastupdated'], axis=1)
        df_SF1_ART=df_SF1_ART.dropna(axis=1,how='all')
        
        
        # Change label of 'datekey' and 'date to do the Merge
        df_SF1_ART = df_SF1_ART.rename(columns={'datekey': 'date'})

        symbol_Premium=pd.merge(symbol_Basic, df_SF1_ART, on='date',how='outer',sort=True)

        # Data Augmentation to use Fundamental Data for 90 days after ART release
        symbol_Premium=symbol_Premium.fillna(method='ffill')
        
        # Drop Values does not have Fundamental Data
        symbol_Premium=symbol_Premium.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82])
        #symbol_Premium=symbol_Premium.dropna() Previous Version
        
        # Generate a Stock Data File all Features
        #time.sleep(10)  
        #symbol_Premium.to_csv (r'gdrive/My Drive/Colab Notebooks/CS230_PROJECT/stock_data/ETSY_Premium.csv', index = True, header=True )

        return symbol_Premium, True
    except Exception as ex:
        print(ex)
        return None, None



#output dir as a csv 

def download_symbol(symbol, output_dir, retry_count=4):

    stock_csv, didPass = try_download(symbol)
    if didPass:
        #input("Press Enter to continue...")
        save(googlepath, stock_csv, output_dir, '{}.csv'.format(symbol))
    else:
        print('Failed to download {}'.format(symbol))

df = pd.read_csv(f"{googlepath}companylist.csv")
for symbol in df.Symbol:
    my_file = Path(f"{googlepath}stock_data/{symbol}.csv")  # check if already downloaded
#     print(my_file.exists())
    if not my_file.exists():
        print('Downloading {}'.format(symbol))
        download_symbol(symbol, 'stock_data')
    else:
        print(f"Already downloaded {symbol}")




Already downloaded MMM
Already downloaded AOS
Already downloaded ABT
Already downloaded ABBV
Already downloaded ACN
Already downloaded ATVI
Already downloaded AYI
Already downloaded ADBE
Already downloaded AAP
Already downloaded AMD
Already downloaded AES
Already downloaded AET
Already downloaded AMG
Already downloaded AFL
Already downloaded A
Already downloaded APD
Already downloaded AKAM
Already downloaded ALK
Already downloaded ALB
Already downloaded ARE
Already downloaded ALXN
Already downloaded ALGN
Already downloaded ALLE
Already downloaded AGN
Already downloaded ADS
Already downloaded LNT
Already downloaded ALL
Already downloaded GOOGL
'date'
Failed to download GOOG
Already downloaded MO
Already downloaded AMZN
Already downloaded AEE
Already downloaded AAL
Already downloaded AEP
Already downloaded AXP
Already downloaded AIG
Already downloaded AMT
Already downloaded AWK
Already downloaded AMP
Already downloaded ABC
Already downloaded AME
Already downloaded AMGN
Already downloaded

'date'
Failed to download SYMC
Already downloaded SYF
Already downloaded SNPS
Already downloaded SYY
Already downloaded TROW
Already downloaded TTWO
Already downloaded TPR
Already downloaded TGT
Already downloaded TEL
Already downloaded FTI
Already downloaded TXN
Already downloaded TXT
Already downloaded BK
Already downloaded CLX
Already downloaded COO
Already downloaded HSY
Already downloaded MOS
Already downloaded TRV
Already downloaded DIS
Already downloaded TMO
Already downloaded TIF
Already downloaded TWX
Already downloaded TJX
'date'
Failed to download TMK
Already downloaded TSS
Already downloaded TSCO
Already downloaded TDG
Already downloaded TRIP
Already downloaded FOXA
'date'
Failed to download FOX
Already downloaded TSN
Already downloaded USB
Already downloaded UDR
Already downloaded ULTA
Already downloaded UAA
'date'
Failed to download UA
Already downloaded UNP
Already downloaded UAL
Already downloaded UNH
Already downloaded UPS
Already downloaded URI
Already downloaded UTX
